## 2.1 FPN于RPN（来自于Faster RCNN）中的应用
　　RPN是Faster R-CNN中用于区域选择的子网络，在Faster R-CNN中anchor有3种大小scale，宽高3种比率ratio{1:1，1:2，2:1}，共9种anchor框。

　　在FPN中同样用了一个3×3和两个并行的1×1，但是是在每个级上都进行了RPN这种操作。既然FPN已经有不同大小的特征scale了，那么我们就没必要像Faster R-CNN一样采用3中大小的anchor了，只要采用3种比率的框就行了。

　　详细的网络结构如图所示，作者在每一个金字塔层级应用了单尺度的anchor，$\{P2,P3,P4,P5,P6\}$分别对应的anchor尺度为$\{32^2,64^2,128^2,256^2,512^2\}$，当然目标不可能都是正方形，本文仍然使用三种比例$\{1:2,1:1,2:1\}$，所以金字塔结构中共有15种anchors。
  
　　对于$P6$，根据论文中所给的它是对$P5$stride为2的下采样。


<img src="images/FPN_rpn.png" width="900" height="400" align="botton" />

　　从图上看出各阶层共享后面的分类网络。这也是强调为什么各阶层输出的channel必须一致的原因，这样才能使用相同的参数，达到共享的目的。

## 2.2 FPN于Fast RCNN中的应用
　　Fast R-CNN中很重要的是ROI Pooling层，需要对不同层级的金字塔制定不同尺度的ROI。ROI Pooling层使用region proposal的结果和中间的某一特征图作为输入，得到的结果经过分解后分别用于分类结果和边框回归。 

　　然后作者想的是，不同尺度的ROI使用不同特征层作为ROI pooling层的输入，大尺度ROI就用后面一些的金字塔层，小尺度ROI就用前面一点的特征层。为此作者定义了一个系数Pk，其定义为： 

$$k=[k_0+\log_{2}(\sqrt{wh}/224)]$$

　　224是ImageNet的标准输入，$k_0$是基准值，设置为5，代表$P5$层的输出（原图大小就用$P5$层），w和h是ROI区域的长和宽，假设ROI是$112*112$的大小，那么$k=k_0-1=5-1=4$，意味着该ROI应该使用$P4$的特征层。k值应该会做取整处理，防止结果不是整数。 

此时模型的结构如图片所示

<img src="images/FPN_fastrcnn.png" width="900" height="600" align="botton" />